<a href="https://colab.research.google.com/github/flow4u/public/blob/master/Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid-19

See for s**ources used**, more **detailed description of the graphs**, and **updates**: https://github.com/flow4u/public/blob/master/README.md
 
 
To run this notebook log into your Google account and press **CTRL + F9**

## Import Modules & Data



In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import ipywidgets as widgets2
from google.colab import widgets
import plotly.express as px
import math

In [0]:
data_sets = ('Confirmed', 'Recovered', 'Deaths')
base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-"
population_data = "https://raw.githubusercontent.com/flow4u/public/master/Countries%20-%20COVID-19%20countries%20-%20CountriesMapped.csv"
df_population = pd.read_csv(population_data)

df=[]
for set in data_sets:
  df_temp = pd.read_csv(base_url+set+'.csv')
  df_temp = df_temp.drop(['Lat', 'Long'],axis=1)
  del df_temp['Province/State']
  if (set=='Deaths'):
    df_deaths = df_temp
    df_deaths = df_deaths.groupby(['Country/Region']).sum()
    df_tempD = df_deaths.T.reset_index().drop(['index'], axis=1)
    df_tempD.index.names = ['day']
    df_tempD.columns.names = ['']

    for col in df_tempD:
      zeros = (df_tempD[col]<1).sum()
      df_tempD[col] = df_tempD[col].shift(-zeros+1)

    # China first death 2020-01-09
    # dataset starts 2020-01-22
    # China needs to shift: 12 days
    # https://en.wikipedia.org/wiki/Timeline_of_the_2019%E2%80%9320_coronavirus_pandemic_in_November_2019_%E2%80%93_January_2020#17_November_2019


    df_tempD['China'] = df_tempD['China'].shift(12)

    df_tempD = df_tempD.reset_index()
    df_tempD.index.names = ['index']
  # not all date columns have the same format, some are mm/d/yy, others mm/dd/yy
  # this will convert first to date, then back to string
  for col in df_temp.columns:
    try:
      col1 = col[:len(col)-2]+'20'+col[-2:]
      col_new = datetime.strptime(col1,'%m/%d/%Y')
      col_new = datetime.strftime(col_new, '%Y-%m-%d')
      df_temp.rename(columns={col: col_new}, inplace=True)
    except:
      pass
  df_temp = df_temp.groupby(['Country/Region']).sum()
  df_temp['Status'] = set
  try:
    df = df.append(df_temp, sort=False)
  except:
    df=df_temp

df_population.set_index('all_countries', inplace=True)
testing_dit = df_tempD.T
df_normalized = testing_dit.T

for item in df_normalized:
  if not(item=='day'):
    divider1 = df_population.loc[item, 'Population2020']
    # divider2 = df_population.loc[item, 'P-KM2']
    if math.isnan(divider1):
      df_normalized[item] = 0
      # df_pop_density_normalized[item] = 0
    else:
      df_normalized[item] = df_normalized[item]/divider1*100000
      # df_normalized[item] = df_normalized[item]/divider2

df2 = df 
df_status = df2.groupby(['Status']).sum()

df = df.reindex(sorted(df.columns), axis=1)
df_plot = df_status.T.reset_index()




## Filters

When a change is made, use **CTRL + F10**

In [0]:
europe = ['Austria', 'Belgium', 'Croatia', 'Finland', 'France',
          'Germany','Italy', 'Norway', 'Romania','Greece', 'Spain', 'Sweden',
          'Switzerland', 'North Macedonia', 'Denmark', 'Netherlands',
          'San Marino', 'Iceland', 'Luxembourg', 'Monaco',
          'Andorra', 'Portugal', 'Hungary',
          'Liechtenstein', 'Bosnia and Herzegovina', 'Slovenia','Poland',
          'Lithuania','Georgia','Estonia', 'Latvia', 'Belarus',
          'Armenia', 'Ukraine', 'Serbia', 'Slovakia','Kosovo', 
          'Montenegro', 'Guernsey', 'Jersey',
          'Bulgaria', 'Malta', 'Albania', 'Cyprus',
          'Holy See', 'Ireland',
          'Czechia', 'Moldova', 'Russia',
          'United Kingdom', 'Turkey']

asia = ['Cambodia', 'India', 'Azerbaijan', 
        'Japan', 'Malaysia', 'Nepal', 'Philippines', 
        'Singapore', 'Sri Lanka', 'Thailand',
        'Indonesia', 'Afghanistan', 'Bahrain','Kuwait','Oman',
        'Iraq','Pakistan', 'Israel','Jordan', 'Lebanon', 'Qatar',
        'Saudi Arabia', 'United Arab Emirates', 'Bhutan',
        'Bangladesh', 'Maldives', 'Brunei',
        'Mongolia', 'Kazakhstan', 'Uzbekistan',
        'China', 'Kyrgyzstan', 'East Timor', 'Syria', 'Timor-Leste',
        'Iran', 'Korea, South', 'Taiwan*', 'Vietnam']

north_america = ['US', 'Canada', 'Mexico', 'Martinique', 
                 'Panama', 'Honduras', 'Jamaica', 'Antigua and Barbuda', 
                 'Bahamas, The', 'El Salvador', 'Nicaragua', 'Greenland',
                 'Guatemala', 'Saint Lucia','Haiti', 'Guadeloupe',
                 'Saint Vincent and the Grenadines', 'Puerto Rico',
                 'Barbados', 'Dominica', 'Grenada', 'The Bahamas']

south_america = ['Brazil', 'Dominican Republic', 'French Guiana',
                 'Argentina', 'Chile', 'Ecuador', 
                 'Colombia', 'Costa Rica', 'Peru', 'Togo',
                 'Paraguay', 'Bolivia', 'Cuba', 'Guyana', 'Trinidad and Tobago',
                 'Uruguay', 'Venezuela', 'Suriname']

africa = ['Nigeria', 'Morocco', 'Senegal', 'Algeria', 'Egypt', 'Tunisia',
          'South Africa', 'Cameroon', 'Burkina Faso', 'Congo (Kinshasa)',
          "Cote d'Ivoire", 'Ethiopia', 'Kenya', 'Sudan', 'Guinea',
          'Eswatini', 'Gabon', 'Ghana','Mauritania', 'Namibia', 'Rwanda',
          'Seychelles', 'Central African Republic', 'Congo (Brazzaville)',
          'Equatorial Guinea', 'Benin', 'Liberia', 'Mayotte', 
          'Somalia', 'Tanzania', 'Chad', 'Angola', 'Cabo Verde',
          'Djibouti', 'Gambia, The', 'Mauritius', 'Zambia', 'Madagascar',
          'Niger', 'Zimbabwe','Cape Verde', 'Eritrea', 'Uganda', 'Mozambique',
          'Republic of the Congo', 'The Gambia', 'Reunion', 'test'
          ]

oceania = ['Australia', 'New Zealand', 'Fiji', 'Papua New Guinea', 'Guam']

others = ['Cruise Ship']

continents = {'europe': europe, 'asia': asia, 'north_america': north_america,
              'south_america': south_america,'africa': africa,
              'oceania': oceania, 'others':others}

all_countries=[]
all_countries.extend(others)
all_countries.extend(europe)
all_countries.extend(asia)
all_countries.extend(africa)
all_countries.extend(south_america)
all_countries.extend(north_america)
all_countries.extend(oceania)
# for t in all_countries:
#   print(t)
# print(all_countries)
missing_countries = 'Missing: '
for t in df.index.unique():
  if all_countries.count(t) == 0:
    missing_countries = missing_countries + ", '" + t + "'"


temp_countries = pd.Series([i for i in df.index])
temp_countries = temp_countries.unique()
obsolete_countries = 'Obsolete: '
for country in all_countries:
  if (country not in temp_countries):
    pass
    all_countries.remove(country)
for continent in continents:
  countries = continents[continent]
  for country in countries:
    if (country not in all_countries):
      continents[continent].remove(country)

# View Graphs
Click on space next to SHOW CODE and press **CTRL+ALT+F** for full screen

*Country names below are as is provided in the data

In [0]:
#@title
if missing_countries != 'Missing: ':
  print(missing_countries)
  print('')
  print('The above countries must be added to the appropiate filters in the code')
  print('The code for the filters can be found above this output')

if obsolete_countries != 'Obsolete: ':
  print('')
  print(obsolete_countries)
  print('')
  print('The above countries must be removed to the appropiate filters in the code')
  print('The code for the filters can be found above this output')

updating = True
country_function_list={}
items=[] # continent, countries, buttons
items_all=[] # countries, buttons
countries_list = []
tb_continents = list(continents.keys())
tb_continent_tabs = widgets.TabBar(tb_continents)
tb_special_buttons = widgets.TabBar(['special buttons'])
tb_graph_tabs = ['Confirmed, Existing, Recovered, and Deaths',
                 'Daily Changes',
                 'Death Rate Comparison',
                 'Estimated Recovery Days',
                 'Estimated Mortality Rate']
tb_graph = widgets.TabBar(tb_graph_tabs)
xuse_log=False
yuse_log=False
normal=False

layout=widgets2.Layout(#max_width='100%',
                          grid_template_columns='repeat(auto-fill,130px',
                       )

def replace_char(a):
  variable_name = a.replace(' ', '')
  variable_name = variable_name.replace('.', '')
  variable_name = variable_name.replace('/', '')
  variable_name = variable_name.replace('(', '')
  variable_name = variable_name.replace(')', '')
  variable_name = variable_name.replace(',', '')
  variable_name = variable_name.replace("'", '')
  variable_name = variable_name.replace('*', '')
  variable_name = variable_name.replace('-', '')
  return variable_name

def button_toggled(value):
  global updating
  if(value.owner.description=='Ynormal/log'):
    global yuse_log
    yuse_log = value.new
  if(value.owner.description=='Xnormal/log'):
    global xuse_log
    xuse_log = value.new
  if(value.owner.description=='normal/per 1000'):
    global normal
    normal = value.new
  if(value.owner.description[0:6]=='toggle'):
    updating=False
    update_continent = value.owner.description[7:]
    for i, continent in enumerate(continents):
      if (continent == update_continent):
        for subitem in items[i]:
          exec(replace_char(subitem.description)+'.value='+str(value.owner.value))
    updating=True
  if (updating): update_graphs()


def button_creation(name):
  button_created = widgets2.ToggleButton(value=False, description=name, button_style='')
  variable_name = replace_char(name)
  globals()[variable_name] = button_created
  code_i = variable_name + '.observe(button_toggled, "value")'
  exec(code_i)
  return button_created

def create_selection():
  for i, tab_content in enumerate(tb_continents):
    with tb_continent_tabs.output_to(i, select=False):
      items.append([])
      countries = sorted(continents[tab_content])
      items[i] = [button_creation(country) for country in countries]
      ui = widgets2.GridBox(items[i], layout=layout)
      make_countries_list()
      display(ui)
  
  with tb_special_buttons.output_to(0,0):
    special_buttons=[]
    special_buttons.append([])
    special_buttons[0] = button_creation('Ynormal/log')
    special_buttons.append([])
    special_buttons[1] = button_creation('Xnormal/log')
    special_buttons.append([])
    special_buttons[2] = button_creation('normal/per 1000')
    for i, continent in enumerate(continents):
      special_buttons.append([])
      special_buttons[i+3] = button_creation('toggle '+continent)
    ui = widgets2.GridBox(special_buttons, layout=layout)
    display(ui)


def create_graphs():
  countries_list = make_countries_list()
  process(countries_list)
  if (countries_list):
    global country_selection

    with tb_graph.output_to(0, select=False):
      title = 'COVID-19: Showing' + country_selection
      labels_axis = {'Status':'', 'value':'# of people', 'index':'date'}
      category_color = {'Confirmed':'grey',
                        'Deaths':'red',
                        'Existing':'orange',
                        'Recovered':'green',
                        }
      global yuse_log
      df_enhanced=df_plot
      df_enhanced.drop(['Recovered + Deaths',
                        'Confirmed - Deaths',
                        ],
                      axis=1,
                      inplace=True)
      df_simple = df_enhanced.melt(id_vars='index')

      fig = px.line(df_simple, x='index', y='value',
                    color='Status',
                    color_discrete_map=category_color,
                    labels=labels_axis,
                    log_y=yuse_log, 
                    title=title)
      fig.show()

    with tb_graph.output_to(1, select=False):
      title = 'COVID-19: Daily Mutations in' + country_selection
      labels_axis = {'Status':'', 'value':'# of people', 'index':'date'}
      category_color = {'Confirmed':'grey',
                        'Recovered + Deaths':'blue',
                        'Deaths':'red',
                        'Recovered':'green'}

      fig = px.bar(df_delta, x='index', y='value',
                  color='Status',
                  color_discrete_map=category_color,
                  labels=labels_axis,
                  barmode='group', 
                  log_y=yuse_log, 
                  title=title
                  )
      fig.show()

 
    with tb_graph.output_to(2, select=False):
      title = 'COVID-19: Death Progression' + country_selection
      labels_axis = {'country':'', 'deaths':'# of deaths', 'day':'# of days since 1th death'}
      filter = countries_list
      filter.append('day')
      global normal
      if (normal):
        title = 'COVID-19: Death Progression Normalized per 1000 People' + country_selection
        labels_axis = {'country':'', 'deaths':'# of deaths per 1000 People', 'day':'# of days since 1th death'}
        df_simple = df_normalized
      else: 
        df_simple = df_tempD

      df_simple = df_simple[filter].melt(id_vars='day', var_name='country', value_name='deaths')
      fig = px.line(df_simple, x='day', y='deaths',
                  color='country',
                  labels = labels_axis,
                  log_y = yuse_log,
                  log_x= xuse_log, 
                  title=title
                  )
      fig.show()


    with tb_graph.output_to(3, select=False):
      title = 'COVID-19: Estimated Recovery Time in' + country_selection
      labels_axis = {'variable':'', 'value':'# of recovery days', 'index':'date'}
      category_color = {'RecoveryDaysUpper':'green', 'RecoveryDaysLower':'blue'}
      df_simple = df_recovery_time.melt(id_vars='index')
      fig = px.line(df_simple, x='index', y='value',
                    color='variable',
                    color_discrete_map=category_color,
                    labels=labels_axis, 
                    title=title)
      fig.show()

    with tb_graph.output_to(4, select=False):
      title = 'COVID-19: Estimated Mortality Rate of' + country_selection
      labels_axis = {'variable':'', 'value':'%', 'index':'date'}
      category_color = {'Upper estimate':'red', 'Lower estimate':'orange'}

      fig = px.line(df_mortality_rate, x='index', y='value',
                    color='variable',
                    color_discrete_map=category_color,
                    labels=labels_axis, 
                    title=title)
      fig.show()


def update_graphs():
  countries_list = make_countries_list()
  for i, _ in enumerate(tb_graph_tabs):
    with tb_graph.output_to(i, select=False):
      tb_graph.clear_tab()
  create_graphs()

def sub_item_processing(a):
  b = replace_char(a.description)
  global country_function_list
  country_function_list[b] = a
  return b

def process(countries_list):
  if (countries_list):
    global country_selection
    country_selection = ''
    for t, c in enumerate(countries_list):
      if (t%8==0):
        country_selection += '<br>' + c
      elif (t<15):
        country_selection += ', ' + c
      else:
        country_selection += ' .....'
        break

      # creating the different dataframes
    df2 = df[df.index.isin(countries_list)]
    df_status = df2.groupby(['Status']).sum()
    global df_plot
    df_plot = df_status.T.reset_index()
    df_plot['Existing'] = df_plot['Confirmed'] - df_plot['Deaths'] - df_plot['Recovered']
    df_plot['Recovered + Deaths'] = df_plot['Recovered'] + df_plot['Deaths']
    df_plot['Confirmed - Deaths'] = df_plot['Confirmed'] - df_plot['Deaths']
    df_temp2 = df_status.T.reset_index()

    # daily changes: additional confirmed, delta recovered + deaths
    global df_delta
    df_delta = df_plot
    df_delta = df_delta.set_index('index')
    df_delta = df_delta.diff()
    df_delta = df_delta.reset_index()
    df_delta.drop(['Existing',
                   'Confirmed - Deaths',
                   ],
                  axis=1,
                  inplace=True)
    df_delta = df_delta.melt(id_vars='index')

    days_list1=[]
    days_list2=[]
    date_list=[]
    mortality_rate=[]
    for _, row in df_plot.iterrows():
      value1_lookup = row['Recovered']
      value2_lookup = row['Confirmed']
      value3_lookup = row['Deaths']
      if(value3_lookup>0):
      # if(row['Recovered']>0):
        # value1_lookup = row['Recovered']
        # value2_lookup = row['Confirmed']
        # value3_lookup = row['Deaths']
        mortality_rate1 = round(value3_lookup / (value1_lookup+value3_lookup) * 100,2)
        mortality_rate2 = round(value3_lookup / (value2_lookup) * 100,2)
        mortality_rate.append([mortality_rate1, mortality_rate2])
        result_index1 = df_plot['Confirmed - Deaths'].sub(value1_lookup).abs().idxmin()
        result_index2 = df_plot['Confirmed'].sub(value1_lookup+value3_lookup).abs().idxmin()
        date1 = datetime.strptime(row['index'], '%Y-%m-%d')
        date2 = datetime.strptime(df_plot.loc[result_index1]['index'], '%Y-%m-%d')
        date3 = datetime.strptime(df_plot.loc[result_index2]['index'], '%Y-%m-%d')
        date_list.append(row['index'])
        days_list1.append((date1-date2).days)
        days_list2.append((date1-date3).days)


    global df_recovery_time, df_mortality_rate
    df_recovery_time = pd.DataFrame(days_list1, index=date_list, columns=['Upper Estimate'])
    try:
      df_recovery_time['Lower Estimate'] = pd.DataFrame(days_list2, index=date_list, columns=['Lower Estimate'])
    except:
      pass
    df_mortality_rate = pd.DataFrame(mortality_rate, index=date_list, columns=['Upper estimate','Lower estimate'])
    df_recovery_time = df_recovery_time.reset_index()
    df_mortality_rate = df_mortality_rate.reset_index()
    df_mortality_rate = df_mortality_rate.melt(id_vars='index')



def make_countries_list():
  countries_list = []
  for item in items:
    for sub_item in item:
      if (sub_item.value == True):
        countries_list.append(sub_item.description)
  return countries_list

create_selection()
create_graphs()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>